## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,51.80,62,75,10.36,light rain
1,1,Saint George,US,37.1041,-113.5841,71.01,9,1,3.44,clear sky
2,2,Zlobin,HR,45.2924,14.6541,39.20,41,0,5.75,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,50.00,81,90,8.05,overcast clouds
4,4,Hilo,US,19.7297,-155.0900,69.80,83,90,6.91,light rain


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Atuona,PF,-9.8000,-139.0333,80.67,79,68,10.60,broken clouds
9,9,Bambous Virieux,MU,-20.3428,57.7575,77.00,94,40,6.91,scattered clouds
12,12,Candolim,IN,15.5181,73.7626,84.20,84,40,2.30,scattered clouds
14,14,San Patricio,US,28.0170,-97.5169,81.00,69,75,28.77,broken clouds
15,15,Huarmey,PE,-10.0681,-78.1522,75.49,73,96,11.34,overcast clouds
16,16,Cabo San Lucas,MX,22.8909,-109.9124,79.00,60,1,11.50,clear sky
22,22,Ridgecrest,US,35.6225,-117.6709,78.80,11,1,4.61,clear sky
25,25,Vaini,TO,-21.2000,-175.2000,77.00,83,75,6.91,broken clouds
27,27,Kapaa,US,22.0752,-159.3190,77.00,50,40,21.85,scattered clouds
33,33,Rikitea,PF,-23.1203,-134.9692,76.86,72,9,10.04,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Atuona,PF,80.67,broken clouds,-9.8000,-139.0333,
9,Bambous Virieux,MU,77.00,scattered clouds,-20.3428,57.7575,
12,Candolim,IN,84.20,scattered clouds,15.5181,73.7626,
14,San Patricio,US,81.00,broken clouds,28.0170,-97.5169,
15,Huarmey,PE,75.49,overcast clouds,-10.0681,-78.1522,
16,Cabo San Lucas,MX,79.00,clear sky,22.8909,-109.9124,
22,Ridgecrest,US,78.80,clear sky,35.6225,-117.6709,
25,Vaini,TO,77.00,broken clouds,-21.2000,-175.2000,
27,Kapaa,US,77.00,scattered clouds,22.0752,-159.3190,
33,Rikitea,PF,76.86,clear sky,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.count()

City                   197
Country                197
Max Temp               197
Current Description    197
Lat                    197
Lng                    197
Hotel Name             197
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig